In [1]:
%load_ext autoreload
%autoreload 2

## Load Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('../../')
print(f'cwd: {os.getcwd()}')
import torch
import numpy as np
from src.model import predict

cwd: /Users/gbemidebe/Documents/GitHub/monsur


In [3]:
# set plotting style
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.dpi'] = 200
fig_size = (8, 4)

## Data

In [4]:
feature_data = pd.read_csv('./data/cleaned_data.csv')
feature_data.drop(columns=['S/N', 'C2H4', 'CO', 'H2', 'EtoH', 'FORM'], inplace=True)
feature_data.head()

,cDen,Pot,Sn %,pH,weight,Cu %
0,150,3.5,1.00,14.05,118.7100,0.00
1,150,3.3,0.80,14.05,107.6772,0.20
2,150,3.2,0.50,14.05,91.1280,0.50
3,150,3.1,0.10,14.05,69.0624,0.90
4,150,3.0,0.05,14.05,66.3042,0.95


In [5]:
feature_data = feature_data[['cDen', 'Pot', 'Sn %', 'pH', 'weight', 'Cu %']]
feature_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cDen,35.0,269.171429,119.205824,141.000,150.00000,250.0000,350.0000,450.00
Pot,35.0,3.860000,0.500118,2.800,3.55000,4.0000,4.1500,4.70
Sn %,35.0,0.354286,0.388203,0.000,0.03000,0.1000,0.8000,1.00
pH,35.0,12.844000,2.447214,8.020,14.05000,14.0500,14.0500,14.05
weight,35.0,83.089817,21.414838,63.546,65.20092,69.0624,107.6772,118.71
Cu %,35.0,0.645714,0.388203,0.000,0.20000,0.9000,0.9700,1.00


In [6]:
output_data = pd.read_csv('./data/cleaned_data.csv')
output_data.drop(columns=['S/N', 'cDen', 'Pot', 'Sn %', 'pH', 'weight', 'Cu %', 'C2H4', 'CO'], inplace=True)
output_data.head()

,H2,EtoH,FORM
0,0.12,0.00,0.61
1,0.07,0.00,0.66
2,0.05,0.03,0.52
3,0.05,0.02,0.42
4,0.05,0.10,0.19


In [7]:
# normalize the data with max values
normalize_data = False
if normalize_data:
    feature_data['Cu %'] /= 1.00
    feature_data['weight'] /= 118.71
    feature_data['Pot'] /= 4.70
    feature_data['pH'] /= 14.05
    feature_data['cDen'] /= 450.00
# rerrange the columns
feature_data.head()

,cDen,Pot,Sn %,pH,weight,Cu %
0,150,3.5,1.00,14.05,118.7100,0.00
1,150,3.3,0.80,14.05,107.6772,0.20
2,150,3.2,0.50,14.05,91.1280,0.50
3,150,3.1,0.10,14.05,69.0624,0.90
4,150,3.0,0.05,14.05,66.3042,0.95


In [8]:
# input = torch.from_numpy(feature_data.values).float()
# out = predict(input, layer_model=[6, 20, 20, 15, 3], dir='./neuralnetwork/feh')
# out  = out.detach().numpy()
# out

## Optimization

In [9]:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

from pymoo.core.problem import ElementwiseProblem
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
import numpy as np
from src.utils import cu_fraction, get_weight

### 1. Single Objective

In [10]:
class reductionsingle(ElementwiseProblem):

    """
    Problem definition for the optimization problem
        maximize the C2H5OH production, HCOOH production and minimize the H2 production
    Args:
        model: model to be optimized
        xl: lower bounds of the variables ('Sn %', 'Pot', 'pH', 'cDen')
        xu: upper bounds of the variables
    retur"""
    def __init__(self):
        super().__init__(n_var=4, n_obj = 1, n_ieq_constr = 3, vtype=float)

        self.xl = np.array([0.00, 2.80, 8.02, 141.00 ])
        self.xu = np.array([100.00, 4.70, 14.05, 450.00])

    def model(self, df):
        """
        compute the output of the model
        Args:
            df: input data
        Returns:
            out: output of the model
        """     
        input = torch.from_numpy(np.asarray(df)).float()
        out = predict(input, layer_model=[6, 20, 20, 15, 3], dir='./neuralnetwork/feh')

        return out
    
    def _evaluate(self, X, out):
        """
        evaluate the objective functions and constraints
        Args:
            X: input data with form ('Sn %', 'Pot', 'pH', 'cDen') 
            out: output of the model
        Returns:
            out: dictionary of the objective functions and constraints
        """
        # 
        cu = cu_fraction(X[0]/100) # 1 - Sn 
        weight = get_weight(X[0]/100) # Sn * wt of Sn + cu * wt of cu
        # features are arranged in the order of ['cDen', 'Pot', 'Sn %', 'pH', 'weight', 'Cu %']
        features = np.asarray([X[3]/450.00, X[1]/4.70, X[0]/100.00,  X[2]/14.05, weight/118.71, cu/1.00]) 
        # objective functions
        f = self.model(features) # HCOOH, C2H5OH, H2
        # constraints 
        g1 = (f[0] + f[1] + f[2]) - 100
        g2 = -X[0] + 3 # Sn wt% >= 3%,
        #g3 = -X[3] + 320 # cDen >= 300, -cDen <= -300
        g4 = -f[1] + 0.5 # F[1] >= 0.5, -F[1] <= -0.5
  
        out["F"] = [-f[1]] # maximize HCOOH & C2H5OH and minimize H2
        out["G"] = [g1, g2, g4]

In [11]:
problem = reductionsingle()

algorithm = GA(pop_size=100,
    n_offsprings=100,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               get_termination("n_gen", 500),
               seed=1,
               save_history=True,
               verbose=True)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.1444528401 |  0.5200476560 |             - |             -
     2 |      200 |  0.1316702664 |  0.4419126597 |             - |             -
     3 |      300 |  0.1295203567 |  0.3884461534 |             - |             -
     4 |      400 |  0.1001548469 |  0.3002894509 |             - |             -
     5 |      500 |  0.0017456710 |  0.1917467466 |             - |             -
     6 |      600 |  0.0008065701 |  0.1192018276 |             - |             -
     7 |      700 |  0.000000E+00 |  0.0710307402 | -5.034748E-01 | -5.066127E-01
     8 |      800 |  0.000000E+00 |  0.0336721201 | -5.035967E-01 | -5.066434E-01
     9 |      900 |  0.000000E+00 |  0.0130857106 | -5.056844E-01 | -5.231143E-01
    10 |     1000 |  0.000000E+00 |  0.0032466765 | -5.064737E-01 | -5.246994E-01
    11 |     1100 |  0.000000E+00 |  0.0005937053 | -5.073451E-01 | -5.281373E-01
    12 |     120

In [12]:
print(f"Best solution found:") 
print(f"Sn = {res.X[0]:.2f} %, Potential = {res.X[1]:.2f}, pH = {res.X[2]:.2f}, Current Density = {res.X[3]:.2f}")
print(f"FE = {-1*res.F[0]*100:.2f} %")
print(f"j*FE = {res.X[3]*-1*res.F[0]:.2f}")

Best solution found:
Sn = 3.00 %, Potential = 3.64, pH = 14.05, Current Density = 290.66
FE = 54.18 %
j*FE = 157.49
